In [ ]:
import sys
import pandas as pd

import numpy as np
import skopt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import f1_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sklearn
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
import joblib

In [ ]:
training=pd.read_csv("/path_to_ADNI_training_dataset.csv")
test=pd.read_csv("/path_to_ADNI_test_dataset.csv")

In [ ]:
datasetAIBL=pd.read_csv("/path_to_AIBL_dataset.csv")
datasetOASIS=pd.read_csv("/path_to_OASIS_dataset.csv")

In [ ]:
training.rename(columns={ "Unnamed: 0": "PTID" }, inplace = True)
test.rename(columns={ "Unnamed: 0": "PTID" }, inplace = True)
datasetAIBL.rename(columns={ "Unnamed: 0": "PTID" }, inplace = True)
datasetOASIS.rename(columns={ "Unnamed: 0": "PTID" }, inplace = True)

In [ ]:
training=training.replace([np.inf, -np.inf], np.nan)
test=test.replace([np.inf, -np.inf], np.nan)
datasetAIBL=datasetAIBL.replace([np.inf, -np.inf], np.nan)
datasetOASIS=datasetOASIS.replace([np.inf, -np.inf], np.nan)

In [ ]:
training=training.dropna(axis=1, how='all')

In [ ]:
#TODO CHANGE IF NECESSARY
featureSetSelected="MRI+SocioDemography+Genetics+kogTest"
model="rf"
selectMethod="no"

In [ ]:
clf = joblib.load('/path_to_save_model/model_'+featureSetSelected+"_"+model+"_"+selectMethod+'.pkl') 

In [ ]:
import dalex as dx
from sklearn.dummy import DummyClassifier
from sklearn import preprocessing

Y_train=pd.get_dummies(training.DX,drop_first=True).to_numpy().squeeze()
X_train=training.drop(["DX","separation"],axis=1).to_numpy()

scaler = preprocessing.StandardScaler().fit(X_train)
X_train_pre=scaler.transform(X_train)

dummy_clf = DummyClassifier(strategy="most_frequent")
dataNamed=pd.DataFrame(X_train_pre,columns=training.drop(["DX","separation"],axis=1).columns)

In [ ]:
exp = dx.Explainer(dummy_clf, dataNamed, Y_train)

asp = dx.Aspect(exp)


aspects=asp.get_aspects(h=0.5)
aspects

In [ ]:
import warnings
warnings.filterwarnings('ignore')
exp = dx.Explainer(clf, training.drop(["DX","separation"],axis=1), training.DX)

asp = dx.Aspect(exp)


df=pd.DataFrame(columns=aspects.keys())
dataGes=dataGes.append(datasetAIBL, ignore_index=True)


for i in range(dataGes.shape[0]):
    print("Processing subject "+str(i) + " of "+ str(dataGes.shape[0]))
    mai_asp_shap = asp.predict_parts(dataGes.drop(["PTID","DX"],axis=1).iloc[i],variable_groups=aspects, label='for aspects created by user', type='shap', random_state=42,N=1000)

    res=mai_asp_shap.result
    df=df.append(pd.DataFrame([res.importance.tolist()],columns=res.aspect_name))

df.to_csv('/path_to_save_shap_values_for_aspects/shap_values_aspects'+featureSetSelected+"_"+model+"_"+selectMethod+'.csv')

dfGes=pd.DataFrame(np.array([df.abs().sum().index.tolist(),df.abs().sum().tolist()]).T.tolist(),columns=["feature","SHAP_Importance"])


dfGes.to_csv('/path_to_save_shap_values_for_aspects/shap_values_aspects_summarized_per_feature_'+featureSetSelected+"_"+model+"_"+selectMethod+'.csv')